# 1주차 과제
- 파이토치 시작하기 및 간단한 코드 빈칸 채워넣기

## 파이토치 시작하기

In [63]:
import torch
from torch import Tensor

- tensor 개념 파악 (array of numerical values(vector, matrix..))
- 4x3 행렬 두 개 만들고 행으로 병합, 열로 병합


In [64]:
X= torch.ones(4,3)
Y= torch.zeros(4,3)

# 행으로 병합 코드
xy1 = torch.cat((X,Y), dim=1)
print(xy1)
# 열로 병합 코드
xy1 = torch.cat((X,Y), dim=0)
print(xy1)

tensor([[1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


### Linear Regression Implementation from Scratch
- 머신러닝 기초와 비슷한 내용이지만 파이토치로 실행해보며 익숙해지는것이 목표


In [65]:
pip install d2l

In [66]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [67]:
# 데이터 생성
def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([3, -1.5])
true_b = 5.5
features, labels = synthetic_data(true_w, true_b, 2000)

In [68]:
print('features:', features[0],'\nlabel:', labels[0])
# X features 2개 / label 1개

features: tensor([-0.9872, -0.1134]) 
label: tensor([2.7161])


In [69]:
# 데이터 iteration
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, len(indices), batch_size):
        # 데이터 라벨을 통해 batch size 만큼씩 iteration data 나눠 tensor 형식으로
        # 마지막 남은 데이터가 batch size보다 적을 경우 적은 양 그대로 사용
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [70]:
batch_size = 20

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break # 첫번째 iteration만 프린트 / break 없애면 전체 다 프린트

tensor([[-0.3067, -0.6522],
        [-0.2115,  0.3584],
        [ 1.6035,  0.3641],
        [ 1.0697,  0.9729],
        [-1.1339,  2.6120],
        [-1.4264, -0.5983],
        [-0.0858,  0.1843],
        [-0.9181,  0.8872],
        [-1.5800, -1.0831],
        [ 0.0758, -0.2173],
        [ 0.1120, -1.5755],
        [ 0.4544, -1.1907],
        [-0.0040, -0.3295],
        [-0.7700, -1.1198],
        [ 0.1542, -1.5203],
        [-1.4756,  1.4484],
        [ 0.1167, -0.9047],
        [-0.1139, -0.3222],
        [-0.2051, -0.5574],
        [-0.3605,  0.5041]]) 
 tensor([[ 5.5557],
        [ 4.3290],
        [ 9.7748],
        [ 7.2470],
        [-1.8181],
        [ 2.1366],
        [ 4.9686],
        [ 1.4021],
        [ 2.3769],
        [ 6.0605],
        [ 8.2136],
        [ 8.6411],
        [ 5.9831],
        [ 4.8720],
        [ 8.2529],
        [-1.0892],
        [ 7.2095],
        [ 5.6159],
        [ 5.7286],
        [ 3.6609]])


In [71]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True) # 가중치
b = torch.tensor([5.0], requires_grad=True)

In [72]:
def linreg(X, w, b):
    """The linear regression model."""
    return torch.matmul(X, w) + b

In [73]:
def squared_loss(y_hat, y):
    """Squared loss."""
    return (y_hat - y)**2

In [74]:
def sgd(params, lr, batch_size):
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
          param -= lr * param.grad / batch_size
          param.grad.zero_()

In [75]:
lr = 0.01
num_epochs = 10
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w,b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.236717
epoch 2, loss 0.005323
epoch 3, loss 0.000221
epoch 4, loss 0.000106
epoch 5, loss 0.000103
epoch 6, loss 0.000103
epoch 7, loss 0.000103
epoch 8, loss 0.000103
epoch 9, loss 0.000103
epoch 10, loss 0.000103


In [76]:
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: tensor([-0.0002,  0.0003], grad_fn=<SubBackward0>)
error in estimating b: tensor([-8.1539e-05], grad_fn=<RsubBackward1>)


\+ cost graph 그리기 (x축 w, y축 loss)

sgd([w1,b], lr, batch_size) 에서  
unsupported operand type(s) for *: 'float' and 'NoneType'   
다음과 같은 에러가 뜹니다. 해결하지 못했습니다...

In [78]:
import matplotlib.pyplot as plt
# w가 두개이므로 w0와 w1 각각 cost graph를 그림

def linreg2(X, w0, w1, b):
    """The linear regression model."""
    w = torch.tensor([w0, w1])
    return torch.matmul(X, w) + b
net2 = linreg2

w0_list = []
w1_list = []
mse0_list = []
mse1_list = []

num_epochs = 3
w1 = torch.tensor([1.0], requires_grad=True)
b = torch.tensor([5.0], requires_grad=True)

for w0 in torch.arange(0.0, 4.1, 0.1):
  for epoch in range(num_epochs):
      for X, y in data_iter(batch_size, features, labels):
          l0 = loss(net2(X, w0, w1, b), y)
          l0.sum().backward()
          sgd([w1,b], lr, batch_size)
  with torch.no_grad():
      train_l = loss(net2(features, w0, w1, b), labels)
  w0_list.append(w0.item())
  mse0_list.append(train_l.sum().detach().item())

plt.plot(w0_list, mse0_list)
plt.ylabel('Loss')
plt.xlabel('w0')
plt.show()

w0 = torch.tensor([1.0], requires_grad=True)
b = torch.tensor([5.0], requires_grad=True)

for w1_list in torch.arange(0.0, 4.1, 0.1):
  for epoch in range(num_epochs):
      for X, y in data_iter(batch_size, features, labels):
          l1 = loss(linreg2(X, w0, w1, b), y)
          l1.sum().backward()
          sgd([w0,b], lr, batch_size)
  with torch.no_grad():
      train_l = loss(net2(features, w0, w1, b), labels)
  w1_list.append(w1.item())
  mse1_list.append(train_l.sum().detach().item())

plt.plot(w1_list, mse1_list)
plt.ylabel('Loss')
plt.xlabel('w1')
plt.show()

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

### Concise Implementation of Linear Regression
- API 사용해보기

In [ ]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([3, -1.5])
true_b = 5.5
features, labels = d2l.synthetic_data(true_w, true_b, 2000)

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 20
data_iter = load_array((features, labels), batch_size)

In [ ]:
next(iter(data_iter))

In [ ]:
from torch import nn

net = nn.Sequential(nn.LazyLinear(1))

In [ ]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

In [ ]:
loss = nn.MSELoss()

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr)

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

In [ ]:
w = net[0].weight
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias
print('error in estimating b:', true_b - b)